In [1]:
import pandas as pd 
import sqlalchemy as sql
import alpaca_trade_api as tradeapi 
import os
from dotenv import load_dotenv
#import hvplot.pandas 
%matplotlib inline 

In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [4]:
#building the API, setting timeframe and daily prices  

alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = 'v2')

tickers = ['USO', 'TSLA', 'SPY']
start_date = pd.Timestamp("2010-06-29", tz="America/New_York").isoformat() #according to yahoo finance tsla issued shares on this date
end_date = pd.Timestamp("2021-10-15", tz="America/New_York").isoformat()

timeframe = "1D"

In [5]:
#creating pandas df with alpaca API

prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
).df


In [6]:
#creating closing prices df from larger prices_df 

closing_prices_df = pd.DataFrame()

for ticker in tickers:
    closing_prices_df[ticker] = prices_df[ticker]["close"]

closing_prices_df.index = closing_prices_df.index.date

closing_prices_df.head()

,USO,TSLA,SPY
2021-05-26,45.210,619.1900,419.04
2021-05-27,45.555,630.6000,419.69
2021-05-28,45.430,625.7100,420.02
2021-06-01,46.330,624.0300,419.63
2021-06-02,46.910,605.0001,420.38


In [7]:
# Working with SQL in order to not ping the Alpaca API too much  
#Creating SQL connection/engine

database_connection_string = 'sqlite:///'
engine = sql.create_engine(database_connection_string)
engine

Engine(sqlite:///)

In [8]:
#shoving closing_prices_df from Alpaca into a SQL Table 

closing_prices_df.to_sql('closing_prices', engine, index= False, if_exists='replace' )


In [9]:
#Code to grab the data from the SQL table into a pandas df to analyze further 

closing_prices_df = pd.read_sql_table ('closing_prices', con=engine)
closing_prices_df.head()

,USO,TSLA,SPY
0,45.210,619.1900,419.04
1,45.555,630.6000,419.69
2,45.430,625.7100,420.02
3,46.330,624.0300,419.63
4,46.910,605.0001,420.38


In [11]:
daily_percent_change_df = closing_prices_df.pct_change().dropna()
daily_percent_change_df

,USO,TSLA,SPY
1,0.007631,0.018427,0.001551
2,-0.002744,-0.007755,0.000786
3,0.019811,-0.002685,-0.000929
4,0.012519,-0.030495,0.001787
5,0.001279,-0.053182,-0.003616
...,...,...,...
95,0.012622,0.007835,-0.007195
96,-0.000890,0.017760,-0.002393
97,0.001069,0.006876,0.003459
98,0.011216,0.008308,0.016869
